_Última modificación: 09 de noviembre de 2024_
# Práctica 3: Identificación de frases clave y resumen automático de texto
**Hernández Jiménez Erick Yael**: 2023630748.

Escuela Superior de Cómputo.

Ingeniería en inteligencia Artificial. 5BV1.

Tecnologías para el Procesamiento de Lenguaje Natural.

## Resumen
El código de la práctica está diseñado para automatizar el proceso de resumen de texto mediante técnicas de Procesamiento de Lenguaje Natural (PLN), implementando múltiples algoritmos de análisis de texto para obtener resúmenes concisos de documentos. A continuación, se describe la funcionalidad de las distintas secciones del código:

1. **Inicialización del Entorno**: Se importan las bibliotecas necesarias, como `NLTK` para la manipulación de texto, `sumy` para implementar modelos de resumen extractivo, `RAKE` para la extracción de frases clave, y `transformers` para utilizar un modelo preentrenado de BERT.

2. **Carga de Documentos**: El código carga manualmente tres archivos de texto, cada uno representando una carta de la novela *Frankenstein*, y los guarda en variables individuales. Esto permite que el contenido de cada archivo se procese y analice de manera independiente o conjunta.

3. **Preprocesamiento de Texto**: Antes de realizar el resumen, se aplican técnicas de normalización y tokenización al texto de cada carta. Se eliminan palabras vacías y se segmenta el texto en oraciones y palabras para facilitar el procesamiento por los algoritmos de resumen.

4. **Aplicación de Algoritmos de Resumen**:
   - **RAKE (Rapid Automatic Keyword Extraction)**: Este método identifica rápidamente las frases más representativas del texto basándose en la frecuencia y coocurrencia de palabras clave.
   - **TextRank y LSA (Latent Semantic Analysis)**: Estos algoritmos, disponibles en la biblioteca `sumy`, generan resúmenes extractivos basados en la estructura del texto y la relevancia de términos dentro del documento.
   - **BERT (Bidirectional Encoder Representations from Transformers)**: El código utiliza el modelo BERT a través de la biblioteca `transformers` para realizar un análisis semántico más profundo, creando resúmenes que comprenden el contexto y significado del texto de manera avanzada.

5. **Generación del Resumen**: Finalmente, el código presenta los resúmenes generados por cada algoritmo. Cada uno selecciona las oraciones o frases que capturan la esencia del contenido original, proporcionando una comparación entre las distintas técnicas y sus resultados en términos de precisión y coherencia.

> NOTA: Para las funciones personalizadas, si se abre el archivo en un editor de código que soporte la funcionalidad, al colocar el puntero sobre el nombre de estas se podrá visualizar el resumen de su función

In [ ]:
import nltk     # Biblioteca con las herramientas utilizadas para la manipulación y procesamiento de los documentos
import math     # Para operaciones matemáticas complejas
import numpy    # Biblioteca auxiliar
from sumy.parsers.plaintext import PlaintextParser          # Para adaptar las cadenas de texto
from sumy.nlp.tokenizers import Tokenizer                   # Para tokenizar (sumy)
from sumy.summarizers.text_rank import TextRankSummarizer   # Para aplicar TextRank
from sumy.summarizers.lsa import LsaSummarizer              # Para LSA
from nltk.tokenize import word_tokenize, sent_tokenize      # Submódulo para tokenizar (nltk)
from rake_nltk import Rake                                  # Algoritmo Rake modificado
from sumy.summarizers.text_rank import TextRankSummarizer   # Para TextRank
from transformers import pipeline                           # Para usar BERT
nltk.download('punkt_tab')  # Recursos auxiliares para NLTK
nltk.download('stopwords')  # Lista con stopwords en distintos idiomas


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mafes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mafes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Generación de cuerpo de documentos
Extrayendo las primeras 3 cartas del libro de Frankstein, desde el enlace [“https://www.gutenberg.org/ebooks/84”](“https://www.gutenberg.org/ebooks/84”), los textos se guardaron manualmente como archivos de texto en la carpeta docs como [Carta_1](./docs/Carta_1.txt), [Carta_2](./docs/Carta_2.txt) y [Carta_3](./docs/Carta_3.txt). A partir de estos, incluimos un enunciado de cada uno.

In [2]:
with open("docs/Carta_1.txt", 'r', encoding="utf-8") as file:   # Se abre el archivo
    carta_1: str = file.read()                                  # Se guarda en una variable de texto
with open("docs/Carta_2.txt", 'r', encoding="utf-8") as file:
    carta_2: str = file.read()
with open("docs/Carta_3.txt", 'r', encoding="utf-8") as file:
    carta_3: str = file.read()

Tokenizamos en enunciados para, luego, escoger un enunciado al azar de cada capítulo y así generar el documento

In [3]:
# Tokenizamos por enunciados y los agregamos a la variable `original` que almacene todo el contenido
original = sent_tokenize(carta_1)
original.extend(sent_tokenize(carta_2))
original.extend(sent_tokenize(carta_3))

corpus: str = ""                        # Inicializamos la cadena vacía
for enunciado in original:              # Por cada enunciado en el cuerpo `original`...
     corpus += str(enunciado) + " "     # la concatenamos al `corpus`
print(corpus)                           # Visualizamos el contenido

 To Mrs. Saville, England. St. Petersburgh, Dec. 11th, 17—. You will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. I arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. I am already far north of London, and as I walk in the streets of Petersburgh, I feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. Do you understand this feeling? This breeze, which has travelled from the regions towards which I am advancing, gives me a foretaste of those icy climes. Inspirited by this wind of promise, my daydreams become more fervent and vivid. I try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. There, Margaret, the sun is for ever visible, its broad disk just skirting t

Con esto, tenemos el corpus sobre el que trabajaremos a lo largo de la práctica

## Normalización de textos general
El flujo que se usará, y su justificación se explicará a continuación:
1. Conversión a minúsculas: para evitar redundancias en el contenido significativo del cuerpo
2. Elimininación de espacios, números, signos de puntuación y el caracter '—': Estos caracteres se encuentra en los 3 archivos originales, siendo de carácter visual para separar diálogos y contextos en las frases. No aporta contenido al proceso de ninguno de los algoritmos por lo que su eliminación reducirá el análisis. Cabe mencionar que el caracter '—' es disntinto de '-', siendo el último relevante para la generación del resumen debido a que cambia el significado de las palabras adyacentes, por lo que se mantiene en el cuerpo.
3. Filtrado de palabras más cortas de 3 letras: En el inglés, y particularmente en estas cartas, las abreviaciones se usan para honorificos, identificadores de nombres u ordinales que no nos interesarán en este cuaderno. 

In [4]:
# Conversión a minúsculas
corpus: str = corpus.lower()

# Tokenizamos por enunciados
enunciados: list[str] = sent_tokenize(corpus)

# Eliminamos los caracteres que no nos interesen de cada documento sin perder la estructura u orden
for i, enunciado in enumerate(enunciados):
# Para cada enunciadp 'enunciado' con índice 'i' en los elementos enumerados de 'documentos'
        enunciados[i] = ''.join([char for char in enunciado if char.isalpha() or char == ' ' or char == '-'])
        # Al elemento [i] lo reasignamos como la unión con el caractér vacío '' por cada caracter 'char'
        # si 'char' es una letra o es un espacio o el guión '-'

# Tokenizamos por palabras
# Inicializamos en vacío la lista que contiene todas las palabras por enunciados
palabras_por_enunciado: list[list[str]] = []
for enunciado in enunciados:                                # Por cada enunciado en los `enunciados`
    palabras_por_enunciado.append(word_tokenize(enunciado)) # Agregamos la lista de tokens a `palabras por enunciado`

for i, palabras in enumerate(palabras_por_enunciado):       # Por cada elemento `i` de `palabras_por_enunciado`: `palabras`...
       # Reasignamos el elemento `i` con una lista que contiene cada `palabra` si la palabra es de 3 o más caracteres
       palabras_por_enunciado[i] = [palabra for palabra in palabras if len(palabra) > 2]   

# Imprimimos los resultados
for enunciado, palabras in zip(enunciados, palabras_por_enunciado):
    print(enunciado)
    print(palabras,"\n")

 to mrs saville england
['mrs', 'saville', 'england'] 

st petersburgh dec th 
['petersburgh', 'dec'] 

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings
['you', 'will', 'rejoice', 'hear', 'that', 'disaster', 'has', 'accompanied', 'the', 'commencement', 'enterprise', 'which', 'you', 'have', 'regarded', 'with', 'such', 'evil', 'forebodings'] 

i arrived here yesterday and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking
['arrived', 'here', 'yesterday', 'and', 'first', 'task', 'assure', 'dear', 'sister', 'welfare', 'and', 'increasing', 'confidence', 'the', 'success', 'undertaking'] 

i am already far north of london and as i walk in the streets of petersburgh i feel a cold northern breeze play upon my cheeks which braces my nerves and fills me with delight
['already', 'far', 'north', 'london', 'and', 'walk', 'the', 'streets', 'pete

## Resumen automático extractivo de texto

### TF-IDF - NLTK
Siguiendo el ejemplo de [TURING](https://www.turing.com/kb/5-powerful-text-summarization-techniques-in-python), aplicaremos el algoritmo TF-IDF:

In [5]:
# Crearemos clases que conserven estos datos para analizarlos posteriormente
class TF_IDF():
    r'''
    Clase que contiene el resumen por TF-IDF de una serie de tokens:
    '''
    def __init__(self, tokens, corpus, lang: str = "english"):
        r'''
        # __init__
        - tokens: Any = iterable con los tokens del documento sobre el que se aplicará el algoritmo TF-IDF
        - lang: str = nombre del lenguaje a partir del cual se eliminarán las stop-words que NLTK tiene por defecto 
        '''
        self.tokens = tokens
        self.corpus = corpus
        self.stopWords: set = nltk.corpus.stopwords.words(lang) # Definimos el conjunto de stopwords
        self.tablaFrec: dict = {}   # Inicializamos con un diccionario vacío
        self.pesoEnun: dict = {}    # Inicializamos con un diccionario vacío
        self.totalPesos: int = 0    # Número descriptivo del total de los pesos
        self.promedio: int = 0      # Número descriptivo con el promedio de los pesos
        self.resumen: list = []     # Lista vacía para almacenar las palabras más frecuentes que conforman al resumen
        self.relevancia: float = 0  # Número descriptivo que indica la relevancia del enunciado en el corpus
    
    def calcularTF(self) -> None:
        r'''
        # generarTablaFreq
        Genera la tabla de frecuencias normalizadas del documento
        '''
        # Conteo de frecuencias
        for palabra in self.tokens:        # Por cada palabra en los tokens...
            if palabra in self.stopWords:  # si la palabra se incluyen en las stopwords...
                continue                   # las omitimos y continuamos
            else:                          # de lo contrario...
                if palabra in self.tablaFrec:      # si la palabra ya se incluye en la tabla de frecuencias...
                    self.tablaFrec[palabra] += 1   # aumentamos en 1 el contador
                else:                              # sino...
                    self.tablaFrec[palabra] = 1    # Empezamos el conteo en 1
        # Normalización de frecuencias
        for palabra in self.tablaFrec.keys():      # Por cada frecuencia de las palabras...
            self.tablaFrec[palabra] /= len(self.tokens) # las reasignamos siendo divididas por el total de palabras

    def calcularIDF(self) -> None:
        r'''
        # calcularIDF
        Calcular el peso que tiene cada palabra en los enunciados con respecto a todos los documentos
        '''
        num_docs = len(self.corpus)
        for palabra in self.tablaFrec:
        # Por cada palabra en la tabla de frecuencias...
            doc_count = sum(1 for documento in self.corpus if palabra in documento)
            # Aumentamos en uno el contador de documentos que contengan a la palabra de la iteración
            self.pesoEnun[palabra] = math.log(num_docs / (1 + doc_count)) # Se agrega el 1 para suavizar el peso de palabras raras
            # Para luego calcular el valor correspondiente al IDF y asignarlo al diccionario con las palabras del corpus

    def imprimir_tabla(self, tabla: dict) -> None:
        r'''
        # imprimir_tabla
        Imprime la tabla indicada (ya sea la tabla de frecuencias de palabras `tablaFrec` o de documentos `pesoEnun`)
        '''
        # Por cada elemento en la tabla
        for key in tabla:
            # Imprimimos el elemento y su valor
            print(f"'{key}':{tabla[key]}")
        print("\n")

    def calcular_TF_IDF(self) -> None:
        r'''
        # calcular_TF_IDF
        Aplica el algoritmo TF-IDF y guarda el resumen en el atributo `resumen`: lista con las frases más relevantes del documento
        '''
        # Calculamos la tabla de frecuencias de los términos
        self.calcularTF()
        # Calculamos la tabla de frecuencias de los documentos
        self.calcularIDF()
        # Inicializamos el diccionario vacío de los resultados del algoritmo
        tf_idf = {}
        # Establecemos la relevancia del enunciado en 0
        self.relevancia = 0
        # Por cada palabra en la tabla de frecuencias de los términos
        for palabra in self.tablaFrec:
            # Obtenemos la frecuencia del término
            tf = self.tablaFrec[palabra]
            # Obtenemos el peso de la palabra con respecto a todos los documentos
            idf = self.pesoEnun.get(palabra, 0)
            # Calcula la relevancia de la apalabra con el tf-idf del término
            tf_idf[palabra] = tf * idf  # Multiplicación de TF e IDF
            # Adiciona la relevancia del término a la relevancia del enunciado
            self.relevancia += tf * idf
        # General el resumen
        self.resumen = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)

In [6]:
# Aplicamos el algoritmo sobre los enunciados
lista_td_idf: list[TF_IDF] = []
for palabras, enunciado in zip(palabras_por_enunciado, enunciados):
    lista_td_idf.append(TF_IDF(palabras, enunciados))

for elemento in lista_td_idf:
    elemento.calcular_TF_IDF()

# Para cada `enunciado` en `enunciados`, `palabras` en `palabras_por_enunciado` y `elemento`(resumen) en `lista_td_idf`
for enunciado, palabras, elemento in zip(enunciados, palabras_por_enunciado, lista_td_idf):
    # Imprimimos los resultados
    print(enunciado, "\n", palabras, "\n", elemento.resumen, "\n", f"Relevancia: {elemento.relevancia}\n")

 to mrs saville england 
 ['mrs', 'saville', 'england'] 
 [('mrs', 1.1281300877819247), ('saville', 1.1281300877819247), ('england', 0.9929750517458699)] 
 Relevancia: 3.249235227309719

st petersburgh dec th  
 ['petersburgh', 'dec'] 
 [('petersburgh', 1.6921951316728872), ('dec', 1.6921951316728872)] 
 Relevancia: 3.3843902633457743

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings 
 ['you', 'will', 'rejoice', 'hear', 'that', 'disaster', 'has', 'accompanied', 'the', 'commencement', 'enterprise', 'which', 'you', 'have', 'regarded', 'with', 'such', 'evil', 'forebodings'] 
 [('rejoice', 0.2146072338897747), ('disaster', 0.2146072338897747), ('commencement', 0.2146072338897747), ('regarded', 0.2146072338897747), ('forebodings', 0.2146072338897747), ('accompanied', 0.19326696504197657), ('evil', 0.17812580333398811), ('enterprise', 0.15678553448618998), ('hear', 0.13544526563839188)] 
 Relevancia

In [7]:
# Ordenamos `lista_td_idf` de acuerdo a su propiedad `relevancia`
lista_td_idf = sorted(lista_td_idf, key=lambda elemento: elemento.relevancia, reverse=True)

# Generamos el resumen global con las 5 frases más relevantes del documento...
for i in range(0, 5):
    # Uniendo con espacios las palabras de cada enunciado y su relevancia
    print((" ".join(lista_td_idf[i].tokens)).center(70, " ") + (f"{lista_td_idf[i].relevancia}").center(30, " "))

                   commenced inuring body hardship                           4.07753744390572       
                                 july                                        4.07753744390572       
                           archangel march                                  3.619392077968642       
               yet second step taken towards enterprise                     3.4234631896679355      
                     heaven bless beloved sister                            3.388327352587809       


### Frecuencia de palabras normalizada
De acuerdo con [Matthew Mayo](https://www.kdnuggets.com/2019/11/getting-started-automated-text-summarization.html) y [Dante Sblendorio](https://www.activestate.com/blog/how-to-do-text-summarization-with-python/). Se debe seguir el siguiente algoritmo:
1. Normalizar los documentos.
2.  Por cada palabra en el corpus se cuenta su frecuencia
3.  Se obtiene la frecuencia más alta
4.  Se divide cada frecuencia entre la frecuencia más alta
5.  Por cada enunciado y cada palabra en el corpus, si la palabra se encuentra en el enunciado, se aumenta el conteo del enunciado
7.  Se ordenan los enunciados por su puntación y se obtiene el resumen

Seguiremos este algoritmo a continuación:

In [8]:
class Frecuencias_normalizadas():
    r'''
    # Frecuencias_normalizadas
    Clase que almacena las frecuencias normalizadas de un texto
    '''
    def __init__(self, corpus: list[str], tokens_por_enunciado: list[list[str]], lang: str = "english"):
        # Dado que el documento está en inglés, se coloca como predeterminado, pero puede ser modificado a cualquier otro idioma
        r'''
        # __init__
        - **corpus**: _list[str]_. Lista de enunciados del documento.
        - **tokens_por_enunciado**: _list[list[str]]_. Lista con las listas de tokens o palabras de cada enunciado.
        - **lang**: _str (Opcional)_. Cadena con el nombre completo del idioma en el que está el documento.
        ''' 
        # Agregamos las palabras de todo el cuerpo
        self.corpus: list[str] = corpus
        # Inicializamos un set para no repetir elementos en la bolsa de palabras
        bolsa_palabras: set[str] = []
        # Definimos el conjunto de stopwords
        self.stopWords: set = nltk.corpus.stopwords.words(lang) 
        # Por cada enunciado de la lista de enunciados...
        for enunciado in tokens_por_enunciado:
            # Por cada token en el enunciado...
            for token in enunciado:
                # Si el token es una stopword...
                if token in self.stopWords:
                    # Continuamos con el siguiente token
                    continue
                # De lo contrario...
                else:
                    # Agregamos el token a la bolsa de palabras
                    bolsa_palabras.append(token)
        # Guardamos la bolsa de palabras como una lista
        self.palabras: list[str] = list(bolsa_palabras)
        # Iniciamos los diccionarios con las frecuencias y puntajes en vacío
        self.frecuencias_palabras: dict = {}
        self.puntaje_enunciado: dict = {}
        # Inicializamos la lista de resumen vacía
        self.resumen: list = []
    
    def calcular_frecuencias(self) -> None:
        r'''
        # calcular_frecuencias
        Calcula las frecuencias de las palabras en el documento
        '''
        # Inicializa los diccionarios en vacío
        self.frecuencias_palabras = {}
        self.puntaje_enunciado = {}
        # Inicializamos la lista de resumen vacía
        self.resumen = []
        # Por cada enunciado en el cuerpo...
        for enunciado in self.corpus:
            # Por cada token en la bolsa de palabras...
            for token in self.palabras:
                # Si el token se encuentra ya en el diccionario de frecuencias...
                if token in self.frecuencias_palabras.keys():
                    # Se aumenta su conteo
                    self.frecuencias_palabras[token] += 1
                # De lo contrario...
                else:
                    # Se agrega al diccionario y se inicia su conteo
                    self.frecuencias_palabras[token] = 1
        # Encontramos la frecuencia más alta
        max_frecuencia: int = max(self.frecuencias_palabras.values())
        
        # Reasignamos la frecuencia de cada `palabra`...
        for palabra in self.frecuencias_palabras.keys():
            # Dividiéndola por la frecuencia más alta
            self.frecuencias_palabras[palabra] /= max_frecuencia
        
        '''Elimine los '#' en las siguientes líneas para visualizar las palabras por su frecuencia normalizada'''
        #for key, value in self.frecuencias_palabras.items():
        #    print("Palabra:", f"{key}".center(21, ' '), f".\tFrecuencia: {value:.2f}")

        # Por cada enunciado en el cuerpo...
        for enunciado in self.corpus:
            # Por cada palabra en la bolsa de palabras...
            for word in self.palabras:
                # Si la palabra se encuentra en el enunciado...
                if word in enunciado:
                    # Si la relevancia del enunciado se encuentra ya en el diccionario...
                    if enunciado in self.puntaje_enunciado.keys():
                        # Aumentamos el conteo
                        self.puntaje_enunciado[enunciado] += 1
                    # De lo contrario...
                    else:
                        # Se agrega al diccionario y se inicia su conteo
                        self.puntaje_enunciado[enunciado] = 1
        # Generamos el resumen ordenando los enunciados por su relevancia
        self.resumen = sorted(self.puntaje_enunciado, key=lambda enunciado: self.puntaje_enunciado[enunciado], reverse=True)


In [9]:
# Generamos la clase y aplicamos el algoritmo
normalizadas: Frecuencias_normalizadas = Frecuencias_normalizadas(enunciados, palabras_por_enunciado)
normalizadas.calcular_frecuencias()

# Imprimimos el resumen
print(" Resumen: ".center(100, '-'))
for i, enunciado in enumerate(normalizadas.resumen[0:5]):
    print(f"Enunciado {i+1}:\n{enunciado}")

--------------------------------------------- Resumen: ---------------------------------------------
Enunciado 1:
a youth passed in solitude my best years spent under your gentle and feminine fosterage has so refined the groundwork of my character that i cannot overcome an intense distaste to the usual brutality exercised on board ship i have never believed it to be necessary and when i heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew i felt myself peculiarly fortunate in being able to secure his services
Enunciado 2:
but i have one want which i have never yet been able to satisfy and the absence of the object of which i now feel as a most severe evil i have no friend margaret when i am glowing with the enthusiasm of success there will be none to participate my joy if i am assailed by disappointment no one will endeavour to sustain me in dejection
Enunciado 3:
i accompanied the whale-fishers on several expeditions to the

### RAKE - NLTK
En el ejemplo citado por [Manmohan Singh](https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c):
1. Se importa el algoritmo Rake
2. Se guarda en una variable
3. Se aplica el algoritmo sobre el texto original
4. Se obtiene el resumen
```
>>> from rake_nltk import Rake
>>> rake_nltk_var = Rake()
>>> text = """spaCy is an open-source software library for advanced natural language processing,
written in the programming languages Python and Cython. The library is published under the MIT license
and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
>>> rake_nltk_var.extract_keywords_from_text(text)
>>> keyword_extracted = rake_nltk_var.get_ranked_phrases()
>>> print(keyword_extracted)
['advanced natural language processing', 'software company explosion', 
 'programming languages python', 'source software library', 'mit license',
 'matthew honnibal', 'main developers', 'ines montani', 'library', 'written',
 'spacy', 'published', 'open', 'founders', 'cython']
```
> Singh, M., "Keyword Extraction process in Python with Natural Language Processing(NLP)", Medium. Recuperado en: [_https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c_](https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c)

Se usará el método integrado en NLTK para extraer las palabaras clave y, posteriormente, obtener el resumen con los enunciados que contengan estas palabras clave

In [10]:
# Creamos el constructor
alg_rake: Rake = Rake()
# Para este es necesario juntar los enunciados en un solo texto y lo evaluamos en la función
alg_rake.extract_keywords_from_text(". ".join(enunciados))
# Obtenemos el resumen
palabras_clave = alg_rake.get_ranked_phrases()[:5]
# Imprimimos los resultados
print(f"Frases clave encontradas:\n{palabras_clave}\nFrases completas:")
# Imprimimos las frases completas que incluyen los resultados
for palabras in palabras_clave:
    for enunciado in enunciados:
        if palabras in enunciado:
            print(enunciado, '\n')


Frases clave encontradas:
['voluntarily endured cold famine thirst', 'succeed many many months perhaps years', 'old man decidedly refused thinking', 'cold northern breeze play upon', 'beauty every region hitherto discovered']
Frases completas:
i accompanied the whale-fishers on several expeditions to the north sea i voluntarily endured cold famine thirst and want of sleep i often worked harder than the common sailors during the day and devoted my nights to the study of mathematics the theory of medicine and those branches of physical science from which a naval adventurer might derive the greatest practical advantage 

if i succeed many many months perhaps years will pass before you and i may meet 

but the old man decidedly refused thinking himself bound in honour to my friend who when he found the father inexorable quitted his country nor returned until he heard that his former mistress was married according to her inclinations 

i am already far north of london and as i walk in the s

### TextRank
Para usar TextRank, se decidió usar la función construida de `sumy`

In [11]:
# Inicializamos el constructor
textRank: TextRankSummarizer = TextRankSummarizer()
# Agregamos la lista de stopwords
textRank.stop_words = list(nltk.corpus.stopwords.words("english"))
# Agregamos el constructor temporal para formatear el corpus al tipo de documento necesario para el algoritmo en el idioma inglés
temporal: PlaintextParser = PlaintextParser.from_string(". ".join(enunciados), Tokenizer("english"))
# Aplicamos el algoritmo para obtener las 5 frases más relevantes
resultados = textRank(temporal.document, 5)
# Imprimimos el resumen
for resultado in resultados:
    print(resultado, "\n")

i shall satiate my ardent curiosity with the sight of a part of the world never before visited and may tread a land never before imprinted by the foot of man. 

but i have one want which i have never yet been able to satisfy and the absence of the object of which i now feel as a most severe evil i have no friend margaret when i am glowing with the enthusiasm of success there will be none to participate my joy if i am assailed by disappointment no one will endeavour to sustain me in dejection. 

you may deem me romantic my dear sister but i bitterly feel the want of a friend. 

a youth passed in solitude my best years spent under your gentle and feminine fosterage has so refined the groundwork of my character that i cannot overcome an intense distaste to the usual brutality exercised on board ship i have never believed it to be necessary and when i heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew i felt myself peculiarly 

### BERT - Transformers
El modelo que se usa es el modelo BART que usa Facebook, mismo que está basado en BERT

In [12]:
# Inicializamos el modelo `BART` en el cuaderno
bert: pipeline = pipeline("summarization", model="facebook/bart-large-cnn")

Este constructor, como desventaja, tiene un límite de procesamiento, siendo que el tamaño de nuestro texto supera este límite, por lo que dividiremos en 4 porciones de aproximadamente 3895 caracteres cada uno ($\frac{15579}{4} = 3894.75 \approx 3895$) o hasta el punto más cercano para no cortar las oraciones

In [13]:
# Se muestra la longitud del corpus
print("Longitud total: ",len(". ".join(enunciados)))

Longitud total:  15579


In [14]:
# Inicializamos la lista de porciones en vacío
porciones: list[str] = []
# Agregamos las porciones
porciones.append(". ".join(enunciados)[:3993])
porciones.append(". ".join(enunciados)[3993:7913])
porciones.append(". ".join(enunciados)[7913:11771])
porciones.append(". ".join(enunciados)[11771:])
# Imprimimos las porciones
for i, porcion in enumerate(porciones):
    print(f"Porción {i}:\n{porcion}")

Porción 0:
 to mrs saville england. st petersburgh dec th . you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. i arrived here yesterday and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. i am already far north of london and as i walk in the streets of petersburgh i feel a cold northern breeze play upon my cheeks which braces my nerves and fills me with delight. do you understand this feeling. this breeze which has travelled from the regions towards which i am advancing gives me a foretaste of those icy climes. inspirited by this wind of promise my daydreams become more fervent and vivid. i try in vain to be persuaded that the pole is the seat of frost and desolation it ever presents itself to my imagination as the region of beauty and delight. there margaret the sun is for ever visible its broad disk just skirting the horizon 

In [15]:
# Inicializamos lo resultados en vacío
textos_bert: list[str] = []
# Por cada `porcion`: `i`...
for i, porcion in enumerate(porciones):
    # Agregamos los resultados a la lista
    textos_bert.append(bert(porcion, max_length=200, min_length=30, do_sample=False)[0]['summary_text'])
    # Sobre el modelo `bert`, el la `porcion` de texto será procesada para obtener máximo 200 caracteres o mínimo 30 como resumen parcial
    # y obtenemos el primer elemento del diccionario de resultados con la llave `summary_text`

In [16]:
# Imprimimos los resultados parciales
for i, texto in enumerate(textos_bert):
    print(f"Resultado {i+1}:\n{texto}\n\n")

Resultado 1:
i am already far north of london and as i walk in the streets of petersburgh i feel a cold northern breeze play upon my cheeks which braces my nerves and fills me with delight. i try in vain to be persuaded that the pole is the seat of frost and desolation it ever presents itself to my imagination as the region of beauty and delight. there margaret the sun is for ever visible its broad disk just skirting the horizon and diffusing a perpetual splendour. therefor with your leave my sister i will put some trust in preceding navigatorsthere snow and frost are banished and sailing over a calm sea we may be wafted to a land surpassing in wonders.


Resultado 2:
Six years have passed since i resolved on my present undertaking. i can even now remember the hour from which i dedicated myself to this great enterprise. i accompanied the whale-fishers on several expeditions to the north sea. i voluntarily endured cold famine thirst and want of sleep. and now dear margaret do i not dese

In [17]:
# Volvemos a procesar las porciones en el modelo `bert` con la unión de los resultados parciales
resumen_bert: str = bert(" .".join(textos_bert), max_length=437, min_length=30, do_sample=False)[0]['summary_text']
# Imprimimos el resumen
print(resumen_bert)

i am already far north of london and as i walk in the streets of petersburgh i feel a cold northern breeze play upon my cheeks which braces my nerves and fills me with delight. i try in vain to be persuaded that the pole is the seat of frost and desolation it ever presents itself to my imagination as the region of beauty and delight. with your leave my sister i will put some trust in preceding navigatorsthere snow and frost are banished.


### LSA

In [18]:
# Agregamos el constructor con el modelo
modelo_lsa: LsaSummarizer = LsaSummarizer()
# Aplicamos el algoritmo sobre el documento auxiliar creado anteriormente en la sección de `TextRank` para obtener las 5 frases más relevantes
resumen_lsa: tuple[()] = modelo_lsa(temporal.document, 5)
# Imprimimos los resultados
for i, enunciado in enumerate(resumen_lsa):
    print(f"Enunciado {i+1}:\n{enunciado}\n")

Enunciado 1:
these visions faded when i perused for the first time those poets whose effusions entranced my soul and lifted it to heaven.

Enunciado 2:
i desire the company of a man who could sympathise with me whose eyes would reply to mine.

Enunciado 3:
well these are useless complaints i shall certainly find no friend on the wide ocean nor even here in archangel among merchants and seamen.

Enunciado 4:
shall i meet you again after having traversed immense seas and returned by the most southern cape of africa or america.

Enunciado 5:
this letter will reach england by a merchantman now on its homeward voyage from archangel more fortunate than i who may not see my native land perhaps for many years.

